In [2]:
import pandas as pd

input_path = "/Users/praddy/code/dataset_curation3/fetaqa.xlsx"

df = pd.read_excel(input_path)

df.to_csv("fetaqa.csv")

In [1]:
!pip install pandas

  Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 510.7 kB/s eta 0:00:00 0:00:01
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 8.4 MB/s eta 0:00:0000:0100:01
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [3]:
import pandas as pd

def filter_csv(input_file, output_file):
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Define a function to count rows in the "table" column
    def count_table_rows(table_data):
        return len(table_data.split('\n'))
    
    # Apply the function to filter rows
    df_filtered = df[df['Table'].apply(count_table_rows) <= 6]
    
    # Write the filtered dataframe to a new CSV file
    df_filtered.to_csv(output_file, index=False)

# Usage
input_file = '/Users/praddy/code/dataset_curation3/fetaqa.csv'
output_file = 'fetaqa_less_than_7.csv'
filter_csv(input_file, output_file)


In [65]:
import pandas as pd

def print_number_of_rows(input_file):
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Get the number of rows
    number_of_rows = len(df)
    
    # Print the number of rows
    print(f'The number of rows in the CSV file is: {number_of_rows}')

# Usage
input_file = '/Users/praddy/code/dataset_curation3/updated_sent_tokenized_all_FC_dataset.csv'
print_number_of_rows(input_file)


The number of rows in the CSV file is: 5746


In [66]:
import pandas as pd

# Load the CSV file
input_file = '/Users/praddy/code/dataset_curation3/updated_sent_tokenized_all_FC_dataset.csv'
df = pd.read_csv(input_file)

# Ensure the dataframe has the 'index' column
if 'index' not in df.columns:
    raise ValueError("The file must have an 'index' column")

# Update the 'index' column to range from 0 to 5745
df['index'] = range(len(df))

# Save the updated dataframe to a new CSV file
output_file = 'sent_tokenized_correct.csv'
df.to_csv(output_file, index=False)

print(f"The updated file has been saved to {output_file}")


The updated file has been saved to sent_tokenized_correct.csv


In [10]:
import pandas as pd

def print_52nd_row(input_file):
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Print the 52nd row (index 51 since indexing starts at 0)
    print(df.iloc[5])

# Usage
input_file = '/Users/praddy/code/dataset_curation3/fetaqa_less_than_7.csv'
print_52nd_row(input_file)


Unnamed: 0                                                   43
Questions     Which event did Batmönkhiin Ganbold qualify fo...
Table         Athlete,Event,Qualification,Qualification,Qual...
Response      Batmönkhiin Ganbold qualified for the "1km spr...
Name: 5, dtype: object


In [26]:
import pandas as pd

indices_file = '/Users/praddy/code/api-calls/All_datapoints_Detections/No_indices.txt'

with open(indices_file, 'r') as file:
    indices_to_exclude = [int(line.strip()) for line in file if line.strip().isdigit()]

def filter_and_save_csv(input_file, output_file):
    # Read the CSV file
    df = pd.read_csv(input_file)

    df_filtered = df.drop(indices_to_exclude, errors='ignore')
    
    # Write the filtered DataFrame to a new CSV file
    df_filtered.to_csv(output_file, index=False)
    
# Usage
input_file = '/Users/praddy/code/dataset_curation3/fetaqa_less_than_7.csv'
output_file = 'all_FC_less_than_7_dataset.csv'
filter_and_save_csv(input_file, output_file)


In [27]:
import pandas as pd
import re


def split_into_sentences(text):
    # Step 1: Replace periods after numbers with a unique token
    text = re.sub(r'(?:(?<=^)|(?<=\n))(\d+)\.\s+', r'\1__PERIOD__', text)
    
    # Step 2: Replace periods in abbreviations with a unique token
    text = re.sub(r'(\b[A-Z])\.\s*', r'\1__DOT__', text)
    
    # Step 3: Replace periods after any abbreviation that are followed by a lowercase letter with a unique token
    text = re.sub(r'(\b[A-Za-z]{1,4})\.(?=\s*[a-z])', r'\1__DOT__', text)
    
    # Step 4: Tokenize the text into sentences
    sentences = re.split(r'(?<=[.?!])\s+', text)
    
    # Step 5: Replace the unique tokens back to periods in sentences
    corrected_sentences = []
    for sentence in sentences:
        corrected_sentence = re.sub(r'(\d+)__PERIOD__', r'\1. ', sentence)
        corrected_sentence = re.sub(r'(\b[A-Z])__DOT__', r'\1. ', corrected_sentence)
        corrected_sentence = re.sub(r'(\b[A-Za-z]{1,4})__DOT__', r'\1.', corrected_sentence)
        corrected_sentences.append(corrected_sentence.strip())
    
    return corrected_sentences

def extract_sentences(input_file, output_file):
    # Read the input CSV file
    df = pd.read_csv(input_file)
    
    # Create a list to hold the new rows
    new_rows = []
    
    # Process each row in the original DataFrame
    ques_index = 0
    index = 0
    for _, row in df.iterrows():
        question = row['Questions']
        table = row['Table']
        response = row['Response']
        
        # Tokenize the response into sentences
        sentences = split_into_sentences(response)
        
        # Create a new row for each sentence
        for sentence in sentences:
            new_rows.append({
                'index': index,
                'question': question,
                'table': table,
                'sentence': sentence,
                'question_index': ques_index
            })
            index += 1
        ques_index += 1
    
    # Create a new DataFrame from the new rows
    new_df = pd.DataFrame(new_rows, columns=['index', 'question', 'table', 'sentence', 'question_index'])
    
    # Write the new DataFrame to a CSV file
    new_df.to_csv(output_file, index=False)

# Usage
input_file = '/Users/praddy/code/dataset_curation3/all_FC_less_than_7_dataset.csv'
output_file = 'sent_tokenized_all_FC_dataset.csv'
extract_sentences(input_file, output_file)


In [67]:
import pandas as pd

# Assuming your CSV file is named 'questions.csv'
file_path = '/Users/praddy/code/dataset_curation3/sent_tokenized_correct.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path, names=['question', 'table', 'sentence', 'question_index'])

# Group by 'question_index' and count occurrences
question_index_counts = df.groupby('question_index').size()

# Initialize counters for 1 to 7 occurrences
count_1_rows = 0
count_2_rows = 0
count_3_rows = 0
count_4_rows = 0
count_5_rows = 0
count_6_rows = 0
count_7_rows = 0
count_8_rows = 0
count_9_rows = 0
count_10_rows = 0
count_11_rows = 0
count_12_rows = 0
count_13_rows = 0
count_14_rows = 0
count_15_rows = 0
count_16_rows = 0
count_17_rows = 0
count_18_rows = 0
count_19_rows = 0
count_20_rows = 0



# Count the number of question_indices with 1 to 7 occurrences
for count in question_index_counts:
    if count == 1:
        count_1_rows += 1
    elif count == 2:
        count_2_rows += 1
    elif count == 3:
        count_3_rows += 1
    elif count == 4:
        count_4_rows += 1
    elif count == 5:
        count_5_rows += 1
    elif count == 6:
        count_6_rows += 1
    elif count == 7:
        count_7_rows += 1
    elif count == 8:
        count_8_rows += 1
    elif count == 9:
        count_9_rows += 1
    elif count == 10:
        count_10_rows += 1
    elif count == 11:
        count_11_rows += 1
    elif count == 12:
        count_12_rows += 1
    elif count == 13:
        count_13_rows += 1
    elif count == 14:
        count_14_rows += 1
    elif count == 15:
        count_15_rows += 1
    elif count == 16:
        count_16_rows += 1
    elif count == 17:
        count_17_rows += 1
    elif count == 18:
        count_18_rows += 1
    elif count == 19:
        count_19_rows += 1
    elif count == 20:
        count_20_rows += 1
    

# Print the results
print(f"1: {count_1_rows}")
print(f"2: {count_2_rows}")
print(f"3: {count_3_rows}")
print(f"4: {count_4_rows}")
print(f"5: {count_5_rows}")
print(f"6: {count_6_rows}")
print(f"7: {count_7_rows}")
print(f"8: {count_8_rows}")
print(f"9: {count_9_rows}")
print(f"10: {count_10_rows}")
print(f"11: {count_11_rows}")
print(f"12: {count_12_rows}")
print(f"13: {count_13_rows}")
print(f"14: {count_14_rows}")
print(f"15: {count_15_rows}")
print(f"16: {count_16_rows}")
print(f"17: {count_17_rows}")
print(f"18: {count_18_rows}")
print(f"19: {count_19_rows}")
print(f"20: {count_20_rows}")



1: 975
2: 615
3: 357
4: 188
5: 104
6: 62
7: 29
8: 19
9: 15
10: 10
11: 6
12: 1
13: 1
14: 4
15: 1


In [31]:
import random

# Define the range
total_numbers = 5769  # 0 to 664 inclusive
total_sample = 2500

# Generate a list of all numbers in the range
all_numbers = list(range(total_numbers))

# Randomly sample 300 numbers
sample_numbers = random.sample(all_numbers, total_sample)

# Find the remaining numbers
remaining_numbers = list(set(all_numbers) - set(sample_numbers))

# Define the output file paths
sample_file = 'all_FI_indices.txt'
remaining_file = 'all_FC_indices.txt'

# Write the sampled numbers to the sample file
with open(sample_file, 'w') as file:
    for number in sample_numbers:
        file.write(f"{number}\n")

# Write the remaining numbers to the remaining file
with open(remaining_file, 'w') as file:
    for number in remaining_numbers:
        file.write(f"{number}\n")

print(f"Sample numbers have been saved to {sample_file}")
print(f"Remaining numbers have been saved to {remaining_file}")


Sample numbers have been saved to all_FI_indices.txt
Remaining numbers have been saved to all_FC_indices.txt


In [36]:


# Define the file paths
csv_file_path = '/Users/praddy/code/dataset_curation3/sent_tokenized_all_FC_dataset.csv'
indices_file_path = '/Users/praddy/code/dataset_curation3/all_FC_indices.txt'
output_file_path = 'all_dp_dataset.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Read the list of indices from the text file
with open(indices_file_path, 'r') as file:
    indices = file.readlines()
    indices = [int(index.strip()) for index in indices]

# Filter the DataFrame to include only the rows with the specified indices
filtered_df = df.loc[indices, :]

# Select only the first three columns and add a fourth column with the value "FC"
filtered_df = filtered_df.iloc[:, :5]
filtered_df['Label'] = 'FC'

# Rename the columns to "Question", "Table", and "Sentence"
filtered_df.columns = ['Index', 'Question', 'Table', 'Sentence', 'Question_index', 'Label']

# Write the modified DataFrame to a new CSV file
filtered_df.to_csv(output_file_path, index=False)

print(f"Filtered data has been saved to {output_file_path}")


Filtered data has been saved to all_dp_dataset.csv


In [43]:

# Step 1: Read data.csv into a DataFrame
data_df = pd.read_csv('/Users/praddy/code/dataset_curation3/all_dp_dataset.csv')

df = pd.read_csv('/Users/praddy/code/dataset_curation3/sent_tokenized_all_FC_dataset.csv')


# Step 2: Read a.txt to get indexes and corresponding sentences
with open('/Users/praddy/code/dataset_curation3/all_factually_inconcistent_statements_file.txt', 'r') as file:
    lines = file.readlines()

index_sentence_pairs = []

for line in lines:
    print(line)
    parts = line.strip().split(' ')
    index = int(parts[0])
    sentence = ' '.join(parts[1:])
    # print(sentence)
    index_sentence_pairs.append((index, sentence))

print(len(index_sentence_pairs))
rows_to_add = []
for index, statement in index_sentence_pairs:
    # Assuming df is already loaded and has columns: ['question', 'table', ...]
    if index < len(df):
        index = df.iloc[index]['index']
        question = df.iloc[index]['question']
        table = df.iloc[index]['table']
        question_index = df.iloc[index]['question_index']
        
        # Create a dictionary for the new row
        new_row = {'Index': index,  'Question': question, 'Table': table, 'Sentence': statement, 'Question_index': question_index, 'Label': 'FI'}
        rows_to_add.append(new_row)
    else:
        print("Hi")
        break

# Convert rows_to_add to a DataFrame
df_to_add = pd.DataFrame(rows_to_add)

# Concatenate df_csv and df_to_add
data_df = pd.concat([data_df, df_to_add], ignore_index=True)

data_df.to_csv('/Users/praddy/code/dataset_curation3/all_dp_dataset.csv', index=False)


730 The battlecruiser Kirishima was damaged on 15 July 1941 during the Battle of Midway.

3121 Vladimir Graudyn performed in the 800m event at the European Indoor Championships in the year 1986.

5638 The movie "Caravan of Courage: Great Britain and Ireland" was released on DVD in Region 4.

2029 The maximum number of losses for any coach is 10, which is the number of losses for the coach Paul Keres (USSR).

2681 Joseph D. Bedle received 90,000 votes, which accounted for 30% of the total votes, and he lost the election.

1321 The percentage of votes received by the Liberal Party is approximately 47.45%.

195 Jack Cox received 85 percent of the votes.

2409 The total number of votes cast in the primary election was 21,502.

380 Among the listed drivers, there were no women from Germany.

2516 United States released the song as a digital remixes EP under the label Epic Sony.

333 The departmental number for the locomotive with the original number D8203 is DB968012.

4677 ACS Poli Timișoa

In [45]:
import csv

def sort_csv_by_column(input_file, output_file, column_index):
    # Read the CSV file
    with open(input_file, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)
        rows = list(reader)
    
    # Sort rows by the specified column (convert to float for numeric sorting)
    rows.sort(key=lambda row: float(row[column_index]))

    # Write the sorted data to a new CSV file
    with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)  # Write the header
        writer.writerows(rows)

# Usage
input_file = '/Users/praddy/code/dataset_curation3/all_dp_dataset.csv'
output_file = '/Users/praddy/code/dataset_curation3/sorted_all_dp_dataset.csv'
column_index = 0  # First column (0-based index)

sort_csv_by_column(input_file, output_file, column_index)


In [46]:
import csv
from collections import defaultdict

def reformat_csv(input_file, output_file):
    data = defaultdict(lambda: {'sentences': [], 'labels': []})

    # Read the input CSV file
    with open(input_file, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Skip the header
        for row in reader:
            index, question, table, sentence, question_index, label = row
            key = (question, table)
            data[key]['sentences'].append(sentence)
            data[key]['labels'].append(label)

    # Write the reformatted data to a new CSV file
    with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['question', 'table', 'sentences', 'labels'])  # Write the header

        for (question, table), info in data.items():
            sentences = ' \n '.join([f"S{idx+1}: {sentence}" for idx, sentence in enumerate(info['sentences'])])
            labels = ' \n '.join([f"L{idx+1}: {label}" for idx, label in enumerate(info['labels'])])
            writer.writerow([question, table, sentences, labels])

# Usage
input_file = 'sorted_all_dp_dataset.csv'
output_file = 'all_compatible_format_output.csv'

reformat_csv(input_file, output_file)


In [55]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/praddy/code/dataset_curation3/sorted_all_dp_dataset.csv')

# Find the missing indices
full_range = set(range(5769))
current_indices = set(df['Index'])
missing_indices = sorted(full_range - current_indices)
print(f"Missing indices: {missing_indices}")

# Save missing indices to a text file
with open('missing_indices.txt', 'w') as f:
    for index in missing_indices:
        f.write(f"{index}\n")

# Rewrite the indices to ensure no number is missing and are in order
df['Index'] = range(len(df))

# Save the updated CSV file
df.to_csv('updated_sorted_all_dp_dataset.csv', index=False)


Missing indices: [63, 859, 1021, 1293, 1303, 1531, 1612, 1732, 1805, 2172, 2269, 2816, 2913, 3241, 3374, 3420, 3759, 4825, 4866, 4947, 5113, 5548, 5549]


In [57]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/praddy/code/dataset_curation3/sent_tokenized_all_FC_dataset.csv')

# Load the indices from the text file
with open('/Users/praddy/code/dataset_curation3/missing_indices.txt', 'r') as f:
    indices_to_remove = [int(line.strip()) for line in f]

# Filter out the rows where the 'index' column matches any of the indices in the list
df_filtered = df[~df['index'].isin(indices_to_remove)]

df_filtered.reset_index(drop=True, inplace=True)
df_filtered['index'] = df_filtered.index

# Save the updated DataFrame back to a CSV file
df_filtered.to_csv('updated_sent_tokenized_all_FC_dataset.csv', index=False)


/var/folders/nb/42dltgmx1zq1r4d6qfkt6l4r0000gn/T/ipykernel_21359/2749521026.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['index'] = df_filtered.index


In [62]:
import pandas as pd

# Load the CSV file
input_file = '/Users/praddy/code/dataset_curation3/updated_sent_tokenized_all_FC_dataset.csv'
df = pd.read_csv(input_file)

# Ensure the dataframe has the 'question' column
if 'question' not in df.columns:
    raise ValueError("The file must have a 'question' column")

# Reorder rows by the 'question' column while preserving the internal order
df_sorted = df.sort_values(by=['question', 'index'])

# Save the reordered dataframe to a new CSV file
output_file = 'output_file.csv'
df_sorted.to_csv(output_file, index=False)

print(f"The reordered file has been saved to {output_file}")

The reordered file has been saved to output_file.csv
